In [1]:
#Running this code and the cell below allows us to access files in drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
cd drive/MyDrive/MacAI

/content/drive/MyDrive/MacAI


In [3]:
pip install torchio

     |████████████████████████████████| 143kB 12.4MB/s 
     |████████████████████████████████| 42.5MB 102kB/s 


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from pathlib import Path
from torchio.transforms import HistogramStandardization

In [17]:
import os
import torchio as tio
subjects = []
base_dir = './macai_datasets/brats_new/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData/'

flairs = []
t1s = []
t2s = []
t1ces = []
segs = []

series_strings = ['flair', 't1', 't2', 't1ce', 'seg']
orig_paths = [flairs, t1s, t2s, t1ces, segs]

for file in [file for file in os.listdir('./macai_datasets/brats_new/BraTS2020_TrainingData/MICCAI_BraTS2020_TrainingData') if os.path.isdir(base_dir + file) == True]:
  print(file)

  subject_dict = {}
  for i, _ in enumerate(series_strings):
    current_path =  base_dir+file +'/'+ file+'_' + series_strings[i] +'.nii.gz'
    orig_paths[i].append(current_path)
    if series_strings[i] != 'seg':
      subject_dict[series_strings[i]] = tio.ScalarImage(path=current_path)
    else:
      subject_dict[series_strings[i]] = tio.LabelMap(path=current_path)
    subject = tio.Subject(subject_dict)

  subjects.append(subject)
dataset = tio.SubjectsDataset(subjects)

BraTS20_Training_106
BraTS20_Training_262
BraTS20_Training_203
BraTS20_Training_147
BraTS20_Training_127
BraTS20_Training_349
BraTS20_Training_299
BraTS20_Training_202
BraTS20_Training_181
BraTS20_Training_014
BraTS20_Training_035
BraTS20_Training_177
BraTS20_Training_310
BraTS20_Training_027
BraTS20_Training_083
BraTS20_Training_094
BraTS20_Training_150
BraTS20_Training_092
BraTS20_Training_113
BraTS20_Training_089
BraTS20_Training_061
BraTS20_Training_347
BraTS20_Training_306
BraTS20_Training_216
BraTS20_Training_125
BraTS20_Training_327
BraTS20_Training_277
BraTS20_Training_171
BraTS20_Training_108
BraTS20_Training_308
BraTS20_Training_302
BraTS20_Training_248
BraTS20_Training_321
BraTS20_Training_067
BraTS20_Training_289
BraTS20_Training_128
BraTS20_Training_318
BraTS20_Training_317
BraTS20_Training_104
BraTS20_Training_041
BraTS20_Training_051
BraTS20_Training_303
BraTS20_Training_151
BraTS20_Training_360
BraTS20_Training_131
BraTS20_Training_265
BraTS20_Training_078
BraTS20_Train

In [7]:
landmarks_dict = {}
landmarks_paths = [Path(modality + '_landmarks.npy') for modality in series_strings]

series_strings = ['data', 'seg']
#some serialization problems right now but whatever
for i, series in enumerate(series_strings):
  if series != 'seg':
    print('Training/Loading Nyul for series: '+ series +'\n')
    landmarks_path = landmarks_paths[i]
    landmarks = HistogramStandardization.train(orig_paths[i])
  #  (landmarks_path if landmarks_path.is_file()
  #  else 
  #  )
    torch.save(landmarks, landmarks_path)

    landmarks_dict[series] = landmarks


  0%|          | 0/368 [00:00<?, ?it/s]

Training/Loading Nyul for series: flair



  6%|▌         | 22/368 [00:30<07:25,  1.29s/it]

SystemError: ignored

In [ ]:
print(landmarks_dict)

In [8]:
print(dataset[0])

Subject(Keys: ('flair', 't1', 't2', 't1ce', 'seg'); images: 5)


In [ ]:
# RandomElasticDeformation takes forever, again. Maybe even longer than before.


transforms_dict = {
  tio.HistogramStandardization(landmarks_dict):1,
  tio.transforms.ZNormalization(): 1,
  tio.transforms.RandomGamma(): 0.5,
  #  tio.transforms.RandomElasticDeformation(): 0.5
}
all_transforms = tio.transforms.Compose(transforms_dict)


In [ ]:
iteration =0
for i, _ in enumerate(dataset):
  print(i)

  print(torch.sum(dataset[i]['seg'].tensor))
  transformed = all_transforms(dataset[i]) 
  print(torch.sum(transformed['seg'].tensor))

  iteration += 1
  if iteration >= 10:
    break